# Cloud Data Transfer Speeds Benchmarking Workflow

Add overview of workflow

## Step 0: Load Required Setup Packages & Classes

Installs required workflow setup packages and calls UI generation script. If one or more of the packages don't exist in your `base` environment, they will install for you. Note that if installation is required, this cell will take a few minutes to complete execution.

In [ ]:
import os
import json
import sys
sys.path.insert(0, os.getcwd() + '/utils/jupyter-user-input')
import ui_helpers as ui

## Step 1: Define Workflow Inputs

### Cloud Compute Clusters

In [ ]:
resources = ui.resourceWidgets()
resources.display()

In [ ]:
resources = [{"Name" : "gcptestnew",
              "Controller" : "PW",
              "CSP" : "GCP",
              "Dask Options" : {"Scheduler" : "Slurm",
                                "Partition" : "compute",
                                "Nodes" : 5,
                                "WorkerMemoryGB" : 4
                               }
             }]

### Cloud Object Stores

In [ ]:
storage = ui.storageWidgets()
storage.display()

In [ ]:
storage = [{"URI/Path" : "/home/jgreen/clouddatabenchmarks",
            "CSP" : None
           }]

### Randomly Generated File Options

In [ ]:
randfiles = [{"Format" : "CSV",
             "Generate" : False,
             "SizeGB" : 1
             },
             {"Format" : "NetCDF4",
              "Generate": False,
              "SizeGB" : 1,
              "Data Variables" : 1,
              "Float Coords" : 2,
              "Time Coords" : 1
             },
             {"Format" : "Binary",
              "Generate" : False,
              "SizeGB" : 1
             },
             {"Resource" : resources[0]
             }]

## Step 2: Notebook Setup

In [ ]:
print('Setting up workflow...')

user_input = json.dumps({"RESOURCES" : resources,
             "STORAGE" : storage,
             "RANDFILES" : randfiles})

with open('user_input.json', 'w') as outfile:
    outfile.write(user_input)


#! chmod u+x workflow_notebook_setup.sh
#! ./workflow_notebook_setup.sh

print('Workflow setup complete.')